In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
#LOAD VARIABLES
################################################################################
def LoadData(data_t,approximation):
    horiz_avg=False #not using horizontal average, instead using model average 
    
    #LOADING TERMS
    th=data_t['th'].data
    rv=data_t['qv'].data #called qv in cm1, but is really rv ("mixing ratio")
    rl=data_t['qc'].data+data_t['qr'].data

    if horiz_avg==False:
        th0=data.isel(time=0)['th'][:,0,0].data[:, np.newaxis, np.newaxis]
        rv0=data.isel(time=0)['qv'][:,0,0].data[:, np.newaxis, np.newaxis]
        rl0=data.isel(time=0)['qc'][:,0,0].data[:, np.newaxis, np.newaxis]+data.isel(time=0)['qr'].data[:,0,0][:, np.newaxis, np.newaxis]

    #MAKING MEAN TERMS
    if horiz_avg==True:
        #using horizontal average at each timestep
        th_mean = np.mean(th, axis=(1, 2), keepdims=True)   # shape (z, 1, 1)
        rv_mean = np.mean(rv, axis=(1, 2), keepdims=True)
        rl_mean = np.mean(rl, axis=(1, 2), keepdims=True)

    if horiz_avg==False:
        #using first timestep single column, as in cm1
        th_mean = th0.copy()
        rv_mean = rv0.copy()
        rl_mean = rl0.copy()

    #MAKING PERTURBATION TERMS
    th_prime = th - th_mean
    rv_prime = rv - rv_mean
    rl_prime = rl - rl_mean

    #CALCULATING THETA_V_BAR
    Rd=287.04; Rv=461.5; eps=Rd/Rv #located in cm1/src/constants.F 
    if approximation==True:
        a=((1/eps) - 1) #~0.6077

        if horiz_avg==True:
            #using horizontal average at each timestep
            th_v = th*(1+(a*rv)-rl)
            th_v_mean = np.mean(th_v, axis=(1, 2), keepdims=True)

        if horiz_avg==False:
            #using first timestep single column, as in cm1
            th_v_mean = th0*(1+(a*rv0)-rl0)
        
    # elif approximation==False:
    #     a=((1/eps) - 1) #~0.6077

    #     if horiz_avg==True:
    #         #using horizontal average at each timestep
    #         N=th*(1+(rv/eps)) #numerator
    #         D=(1+rv+rl) #denominator
    #         th_v = N/D
    #         th_v_mean = np.mean(th_v, axis=(1, 2), keepdims=True)

    #     if horiz_avg==False:
    #         #using first timestep single column, as in cm1
    #         N=th0*(1+(rv0/eps)) #numerator
    #         D=(1+rv0+rl0) #denominator
    #         th_v_mean = N/D
    
    return th,th_prime, rv,rv_prime, rl,rl_prime, th_v_mean

In [ ]:
def GetTrueBuoyancy(data, data_t, model=False):
    if model:
        return data_t['buoyancy'].data  # Use model output directly
    
    # Constants
    Rd = 287.04
    Rv = 461.5
    eps = Rd / Rv
    a = (1 / eps) - 1  # ≈ 0.6077
    g = 9.81

    # Reference state (assumes (z,) or (z, 1, 1) for vertical profile)
    th0 = data.isel(time=0)['th'][:, 0, 0].data[:, np.newaxis, np.newaxis]
    rv0 = data.isel(time=0)['qv'][:, 0, 0].data[:, np.newaxis, np.newaxis]
    rl0 = (data.isel(time=0)['qc'][:, 0, 0].data + data.isel(time=0)['qr'][:, 0, 0].data)[:, np.newaxis, np.newaxis]
    
    th_v_mean = th0 * (1 + a * rv0 - rl0)

    # Time-varying fields
    th = data_t['th'].data
    rv = data_t['qv'].data
    rl = (data_t['qc'] + data_t['qr']).data

    th_v = th * (1 + a * rv - rl)

    # Perturbation and buoyancy
    th_v_prime = th_v - th_v_mean
    buoyancy = g * th_v_prime / th_v_mean
    return buoyancy

In [ ]:
def BuoyancyDecompTerms(data_t, approximation):
    #GETTING DATA
    [th,th_prime, rv,rv_prime, rl,rl_prime, th_v_mean] = LoadData(data_t,approximation)

    #CALCULATING VARIABLES
    Rd=287.04; Rv=461.5; eps=Rd/Rv #located in cm1/src/constants.F 
    if approximation==True:
        a=((1/eps) - 1) #~0.6077
        D=th_v_mean

        #MAKING INDIVIDUAL BUOYANCY TERMS (3 TOTAL)
        N=(1+(a*rv)-rl)
        th_term = N/D
        N=(a*th)
        rv_term = N/D
        N=-th
        rl_term = N/D

        #also need to multiply by gravity constant g
        g=9.81 #located in cm1/src/constants.F 
        th_term*=g; rv_term*=g; rl_term*=g

        #also multiply by prime terms with each term
        th_term*=th_prime
        rv_term*=rv_prime
        rl_term*=rl_prime

    # elif approximation==False:
    #     A=(1+(rv/eps))/(1+rv+rl)
    #     D=th_v_mean

    #     #SOME TERM CONSOLIDATIONS
    #     B1=(1-eps+rl)
    #     B2=(eps+rv)
    #     B3=eps*((1+rv+rl)**2)

    #     #MAKING INDIVIDUAL BUOYANCY TERMS (3 TOTAL)
    #     N=A
    #     th_term = N/D
    #     N=th*B1/B3
    #     rv_term = N/D
    #     N=th*B2/B3
    #     rl_term = N/D

    #     #also need to multiply by gravity constant g
    #     g=9.81 #located in cm1/src/constants.F 
    #     th_term*=g; rv_term*=g; rl_term*=g

    #     #also multiply by prime terms with each term
    #     th_term*=th_prime
    #     rv_term*=rv_prime
    #     rl_term*=rl_prime
        

    #CALCULATING BUOYANCY
    buoyancy=GetTrueBuoyancy(data,data_t,model=False)

    #STORING VARIABLES
    VARS={
        'buoyancy': buoyancy,
        'th_term': th_term,
        'rv_term': rv_term,
        'rl_term': rl_term
         }
    return VARS

In [ ]:
def GetOutputName(approximation):
    dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
    # dir2='/mnt/lustre/koa/scratch/air673/'
    if approximation==True:
        out_file = dir2 + 'Variable_Calculation/OUTPUT/' + f'Buoyancy_Decomp_{res}_{t_res}.h5'
    elif approximation==False:
        out_file = dir2 + 'Variable_Calculation/OUTPUT/' + f'Buoyancy_Decomp_FULL_{res}_{t_res}.h5'
    return out_file
    
def initiate_array(VarNames,approximation):
    # Define array dimensions (adjust based on your data)
    t_size = len(data['time'])  # Number of timesteps
    z_size = len(data['zh'])    # Number of vertical levels
    y_size = len(data['yh'])    # Number of y-axis points
    x_size = len(data['xh'])    # Number of x-axis points

    out_file=GetOutputName(approximation)

    with h5py.File(out_file, 'a') as f:
        for var_name in VarNames:
            if var_name not in f:
                f.create_dataset(
                    var_name,
                    shape=(t_size, z_size, y_size, x_size),
                    maxshape=(None, z_size, y_size, x_size),
                    dtype='float64',
                    chunks=(1, z_size, y_size, x_size)
                )

def add_timestep_at_index(VARS, index, approximation):
    out_file=GetOutputName(approximation)
    
    with h5py.File(out_file, 'a') as f:
        for var_name, timestep_data in VARS.items():
            if var_name in f:
                f[var_name][index] = timestep_data
            else:
                raise KeyError(f"Dataset '{var_name}' does not exist in {out_file}")

In [ ]:
#RUNNING

In [ ]:
#MAKING ARRAY TO STORE THETA_E
VarNames=['buoyancy','th_term','rv_term','rl_term']

approximation=True #CHOOSE IF USING FIRST ORDER TAYLOR SERIES APPROXIMATION 
# approximation=False #CHOOSE IF NOT USING ANY APPROXIMATION (data will store will "_FULL" after) (difference is O(1e-6))
initiate_array(VarNames,approximation)

#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in range(len(data['time'])):
    if np.mod(t,1)==0: print(f'Current time {t}')
    data_t=data.isel(time=t)
    
    VARS = BuoyancyDecompTerms(data_t, approximation)
    add_timestep_at_index(VARS, t, approximation)



In [ ]:
##########################################################################
# #READING BACK IN

In [ ]:
# t=100
# approximation=True
# # approximation=False
# in_file=GetOutputName(approximation)
# #READING FINAL OUTPUT
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# # dir2='/mnt/lustre/koa/scratch/air673/'
# with h5py.File(in_file, 'a') as f:
#     # Access the existing dataset 'MSE'
#     th_term = f['th_term'][t]
#     rv_term = f['rv_term'][t]
#     rl_term = f['rl_term'][t]

In [ ]:
##########################################################################
# #TESTING

In [ ]:
from matplotlib.gridspec import GridSpec

# Setup and data loading - unchanged
t = 500

# First dataset (approximation=False)
approximation = True
# approximation = False
in_file = GetOutputName(approximation)
with h5py.File(in_file, 'a') as f:
    th_term = f['th_term'][t]
    rv_term = f['rv_term'][t]
    rl_term = f['rl_term'][t]

z_lev = 5

one = th_term[z_lev]
two = rv_term[z_lev]
three = rl_term[z_lev]

sum_terms=one+two+three

# Second dataset (approximation=True)
approximation = True
in_file = GetOutputName(approximation)
with h5py.File(in_file, 'a') as f:
    buoyancy1 = f['buoyancy'][t] #*#*#
    th_term_approx = f['th_term'][t]
    rv_term_approx = f['rv_term'][t]
    rl_term_approx = f['rl_term'][t]

# Assuming `data` is loaded elsewhere, since you use data.isel(time=t)['buoyancy']
data_t=data.isel(time=t); buoyancy1 = GetTrueBuoyancy(data,data_t,model=True) #*#*#
buoyancy2 = th_term_approx + rv_term_approx + rl_term_approx  # RECONSTRUCTED BUOYANCY

num_levels=20
# Compute min/max for the first row based on sum_terms for consistent color scale
vmin=buoyancy2[z_lev].min();vmax=buoyancy2[z_lev].max()
levels_first_row = np.linspace(vmin, vmax, num_levels)

b1 = buoyancy1[z_lev]
b2 = buoyancy2[z_lev]
diff = b1 - b2

# Shared color levels for buoyancy plots (second row)
vmin_b = min(b1.min(), b2.min())
vmax_b = max(b1.max(), b2.max())
vmax_sym = max(abs(vmin_b), abs(vmax_b))
levels_shared = np.linspace(-vmax_sym, vmax_sym, num_levels)

# Levels for difference plot (symmetric)
diff_max = np.max(np.abs(diff))
levels_diff = np.linspace(-diff_max, diff_max, num_levels)

# Create figure and GridSpec layout 2x4 (4 plots in first row, 2 in second)
fig = plt.figure(figsize=(20, 10))
gs = GridSpec(2, 4, figure=fig, wspace=0.2, hspace=0.3)

# First row: sum_terms first, then th_term, rv_term, rl_term
ax_sum = fig.add_subplot(gs[0, 0])
cs_sum = ax_sum.contourf(sum_terms, levels=levels_first_row, cmap="RdBu_r",extend='both')
ax_sum.set_title('Reconstructed (Sum of Terms)')
ax_sum.set_xticks([])
ax_sum.set_yticks([])

ax_th = fig.add_subplot(gs[0, 1])
cs_th = ax_th.contourf(one, levels=levels_first_row, cmap="RdBu_r",extend='both')
ax_th.set_title('th_term')
ax_th.set_xticks([])
ax_th.set_yticks([])

ax_rv = fig.add_subplot(gs[0, 2])
cs_rv = ax_rv.contourf(two, levels=levels_first_row, cmap="RdBu_r",extend='both')
ax_rv.set_title('rv_term')
ax_rv.set_xticks([])
ax_rv.set_yticks([])

ax_rl = fig.add_subplot(gs[0, 3])
cs_rl = ax_rl.contourf(three, levels=levels_first_row, cmap="RdBu_r",extend='both')
ax_rl.set_title('rl_term')
ax_rl.set_xticks([])
ax_rl.set_yticks([])

# Second row: Model Buoyancy and Difference/Error (span 2 columns each)
ax_buoy = fig.add_subplot(gs[1, 0:1])
cs_buoy = ax_buoy.contourf(b1, levels=num_levels, cmap='RdBu_r')
ax_buoy.set_title('Model Buoyancy')
ax_buoy.set_xticks([])
ax_buoy.set_yticks([])

ax_diff = fig.add_subplot(gs[1, 2:4])
cs_diff = ax_diff.contourf(diff, levels=num_levels, cmap='coolwarm')
ax_diff.set_title('Difference (Model - Reconstructed)')
ax_diff.set_xticks([])
ax_diff.set_yticks([])

# Colorbars — consistent fraction for all
colorbar_fraction = 0.03

# Shared colorbar for first row (4 plots)
cbar1 = fig.colorbar(cs_sum, ax=[ax_sum, ax_th, ax_rv, ax_rl], orientation='vertical', fraction=colorbar_fraction, pad=0.04)
cbar1.set_label('Terms')

# Shared colorbar for second row Model Buoyancy
cbar2 = fig.colorbar(cs_buoy, ax=ax_buoy, orientation='vertical', fraction=colorbar_fraction+0.015, pad=0.04)
cbar2.set_label('Buoyancy')

# Separate colorbar for difference
cbar3 = fig.colorbar(cs_diff, ax=ax_diff, orientation='vertical', fraction=colorbar_fraction, pad=0.04)
cbar3.set_label('Difference')

fig.suptitle(f"Compared Model Buoyancy and Reconstructed Buoyancy Approximation at {data['zh'][z_lev].data*1000:.0f} m")